In [3]:
%load_ext autoreload
%autoreload 2

print('Loading ethanol data')

config_path = "/Users/samsonkoelle/montlake/experiments/configs/ethanol_full.json"
raw_data = "/Users/samsonkoelle/thesis_data/raw_data/ethanol.mat"



Loading ethanol data


In [4]:
import numpy as np
import random
import json
import torch
from convexlocalisometry.vendor import HParams
from convexlocalisometry.atomic_geometry import get_atoms3_full, get_atoms4_full
import itertools

positions = np.load('/Users/samsonkoelle/positions.npy')
randomindices = np.load('/Users/samsonkoelle/indices.npy')

np.random.seed(1234)
random.seed(1234)

config = {}

with open(config_path) as f:
    config.update(json.load(f))


hparams = HParams(**config)


In [120]:
d = hparams.d
n_components = hparams.n_components
atoms2_feat = hparams.atoms2_feat
atoms3_feat = hparams.atoms3_feat
atoms4_feat = hparams.atoms4_feat
atoms2_dict = hparams.atoms2_dict
atoms3_dict = hparams.atoms3_dict
atoms4_dict = hparams.atoms4_dict
diagram = hparams.diagram

ii = np.asarray(hparams.ii)
jj = np.asarray(hparams.jj)
#load geometric features
natoms = positions.shape[1]
n = positions.shape[0]
atoms2 = np.asarray(list(itertools.combinations(range(natoms), 2)))
atoms2full = atoms2
atoms3 = np.asarray(list(itertools.combinations(range(natoms), 3)))
atoms4 = np.asarray(list(itertools.combinations(range(natoms), 4)))
atoms3full = get_atoms3_full(atoms3)
atoms4full = get_atoms4_full(atoms4)

if atoms2_feat:
    atoms2_feats = atoms2full
else:
    atoms2_feats = np.asarray([])

if atoms3_feat:
    atoms3_feats = atoms3full
else:
    atoms3_feats = np.asarray([])

if atoms4_feat:
    atoms4_feats = atoms4full
else:
    atoms4_feats = np.asarray([])

In [10]:
# from pathos.multiprocessing import ProcessingPool as Pool

# #compute rotation/translation invariant featureization
# cores = pathos.multiprocessing.cpu_count() - 1
# pool = Pool(cores)
# print('feature dimensions',atoms2_feats.shape, atoms3_feats.shape,atoms4_feats.shape)
# #import pdb;pdb.set_trace
# results = pool.map(lambda i: get_features(positions[i],
#                            atoms2 = atoms2_feats,
#                            atoms3 = atoms3_feats,
#                            atoms4 = atoms4_feats),
#     data_stream_custom_range(list(range(n))))

# data = np.vstack([np.hstack(results[i]) for i in range(n)])
# np.save('ethanolanglesfull_0524.npy', data)

data = np.load('/Users/samsonkoelle/ethanolanglesfull_0524.npy')

In [13]:
#compute geometry
# radius = hparams.radius
n_neighbors = hparams.n_neighbors
# geom = get_geom(data_svd, radius, n_neighbors)
# geom = get_geom(data, radius, n_neighbors) # megaman doesn't have python 3.10 support
print('next')

next


In [89]:
from sklearn.decomposition import PCA
ind=2001
dimension=2

def local_pca(data,ind,n_neighbors,dimension):
    neighbor_indices = np.linalg.norm(data[:,:] - data[ind,:], axis = 1).argsort()[:n_neighbors]
    local_neighborhood = data[neighbor_indices]
    local_neighborhood = data[neighbor_indices,:]
    centered_data = local_neighborhood - local_neighborhood.mean(axis=0)


    # Perform PCA on the centered neighborhood data
    pca = PCA(n_components=dimension)
    pca.fit(centered_data)
    tangent_bases = pca.components_.transpose()
    return tangent_bases

In [90]:
from convexlocalisometry.atomic_geometry import get_atoms_4
atoms4_dicts= get_atoms_4(natoms, ii, jj)[0]
if atoms2_dict:
    atoms2_dicts = atoms2full
else:
    atoms2_dicts = np.asarray([])
if atoms3_dict:
    atoms3_dicts = atoms3full
else:
    atoms3_dicts = np.asarray([])


In [122]:
atoms4full = get_atoms4_full(atoms4)

In [123]:
atoms4full

array([[0, 1, 2, 3],
       [1, 2, 3, 0],
       [2, 3, 0, 1],
       ...,
       [8, 5, 6, 7],
       [5, 6, 8, 7],
       [6, 5, 7, 8]])

In [91]:
from scipy import sparse
from convexlocalisometry.transformation import exponential_transformation


In [171]:
nsel = 1
selected_points = np.random.choice(range(50000),nsel)
dg_x = np.asarray([get_D_feats_feats(positions[selected_points[i]],
           atoms2in = atoms2_feats,
           atoms3in = atoms3_feats,
           atoms4in = atoms4_feats,
           atoms2out = atoms2_dicts,
           atoms3out = atoms3_dicts,
           atoms4out = atoms4full) for i in range(nsel)])                           
#            atoms4out = atoms4_dicts) for i in range(nsel)])

power = 1
for i, sp in enumerate(selected_points):
    tangent_bases = local_pca(data, sp, n_neighbors, dimension)
    # NOTE (Sam): check convergence rate on weighted local versus local PCA
#     tangent_bases = get_wlpca_tangent_sel(data, geom, replicates[r].selected_points, d)

    # NOTE (Sam): normalization prior to projection once again in the manifold case.
    # or normalize and regress the tangent coordinates themselves in R^B
    dg_x_normalized = exponential_transformation(dg_x[i], power)
    dg_M = np.einsum('b p, b d -> d p', dg_x_normalized, tangent_bases)

In [172]:
from convexlocalisometry.algorithm import greedy, group_basis_pursuit
from convexlocalisometry.loss import isometry_loss

output = greedy(dg_M, isometry_loss, dimension)
print(output)




[658, 621]


In [173]:
dg_M.shape

(2, 756)

In [174]:
np.linalg.svd(dg_M[:,output])[1] 
# NOTE (Sam): shouldn't this always have the best top singular value?
# maybe not necessarily

array([0.89944777, 0.21746936])

In [176]:
print('Computing isometry pursuit solution')
beta = group_basis_pursuit(dg_M)

Computing isometry pursuit solution


/Users/samsonkoelle/miniconda3/envs/isometry/lib/python3.10/site-packages/cvxpy/problems/problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [177]:
basis_pursuit_indices = np.where(np.linalg.norm(beta, axis = 1))[0]
print(basis_pursuit_indices)

[331 639 665]


In [178]:
np.linalg.svd(dg_M[:,basis_pursuit_indices])[1]

array([0.93685615, 0.746045  ])

In [45]:
from convexlocalisometry.atomic_geometry import get_D_feats_feats

In [179]:
basis_pursuit_indices.shape

(3,)

In [180]:
dg_M[:,basis_pursuit_indices].shape

(2, 3)

In [188]:
from convexlocalisometry.algorithm import brute
second_stage_indices = brute(dg_M[:,basis_pursuit_indices], isometry_loss, dimension)

Computing brute force solution for dictionary dimension 3 and target_dimension 2


3it [00:00, 8548.17it/s]


In [189]:
np.linalg.svd(dg_M[:,basis_pursuit_indices[[0,1]]])[1]

array([0.82209478, 0.51852046])

In [187]:
np.linalg.svd(dg_M[:,:])[1]

array([5.98636004, 4.97458501])

In [190]:
brute_indices = brute(dg_M, isometry_loss, dimension)

Computing brute force solution for dictionary dimension 756 and target_dimension 2


0it [00:00, ?it/s]/Users/samsonkoelle/convexlocalisometry/convexlocalisometry/loss.py:14: RuntimeWarning: overflow encountered in exp
  (np.exp(singular_values) + np.exp(singular_values ** (-1))) / (2 * np.e)
285390it [00:10, 28091.77it/s]


In [191]:
np.linalg.svd(dg_M[:,brute_indices])[1]

array([0.78525183, 0.58334542])